In [56]:
import os
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import mnist
import numpy as np
import pandas as pd

In [57]:
save_dir = "save_dir"
save_dir_TFRecord_01 = "save_dir_TFRecord_01"

In [58]:
#save_dir 에 데이터 내려받기
data_sets = mnist.read_data_sets(save_dir, 
                                 dtype=tf.uint8,
                                reshape=False,
                                validation_size=1000)

Extracting save_dir/train-images-idx3-ubyte.gz
Extracting save_dir/train-labels-idx1-ubyte.gz
Extracting save_dir/t10k-images-idx3-ubyte.gz
Extracting save_dir/t10k-labels-idx1-ubyte.gz


In [59]:
print('[0.train     ].image.shape:{}    labels.shape:{}'.format(data_sets[0].images.shape,data_sets[0].labels.shape))
print('[1.validation].image.shape:{}    labels.shape:{}'.format(data_sets[1].images.shape,data_sets[1].labels.shape))
print('[2.test      ].image.shape:{}    labels.shape:{}'.format(data_sets[2].images.shape,data_sets[2].labels.shape))

[0.train     ].image.shape:(59000, 28, 28, 1)    labels.shape:(59000,)
[1.validation].image.shape:(1000, 28, 28, 1)    labels.shape:(1000,)
[2.test      ].image.shape:(10000, 28, 28, 1)    labels.shape:(10000,)


# row 한개씩 추가로 저장하기

In [40]:
# row 한개씩 추가로 저장하기
print_onoff = 0
data_set = data_sets[0]
filename = os.path.join(save_dir_TFRecord_01, 'train_record.tfrecords')
print('filename : {}'.format(filename))
writer = tf.python_io.TFRecordWriter(filename)

#for index in range(data_set.images.shape[0]):
for index in range(10):
    image = data_set.images[index].tostring()
    example = tf.train.Example(features=tf.train.Features(
        feature={
            'height': tf.train.Feature(
                int64_list=tf.train.Int64List(value=[data_set.images.shape[1]])),     # 실데이터가 아닌, image_row의 height Meta값을 저장한다.
            'width': tf.train.Feature(
                int64_list=tf.train.Int64List(value=[data_set.images.shape[2]])),     # 실데이터가 아닌, image_row의 width Meta값을 저장한다.
            'depth': tf.train.Feature(
                int64_list=tf.train.Int64List(value=[data_set.images.shape[3]])),     # 실데이터가 아닌, image_row의 depth Meta값을 저장한다.
            'label': tf.train.Feature(
                int64_list=tf.train.Int64List(value=[int(data_set.labels[index])])),  # label의 실제값을 저장한다.
            'image_raw': tf.train.Feature(
                bytes_list=tf.train.BytesList(value=[image]))}))                      # image의 실제값을 저장하다.
    print_onoff += 1
    if (print_onoff % 10000) == 1:
        print(">>  {} ".format(index))
    writer.write(example.SerializeToString())
writer.close()

print('complete!')

filename : save_dir_TFRecord_01/train_record.tfrecords
>>  0 
complete!


In [41]:
print(example.features.feature["height"].int64_list.value[0])
print(example.features.feature["width"].int64_list.value[0])
print(example.features.feature["depth"].int64_list.value[0])
print(example.features.feature["label"].int64_list.value[0])
#print(example.features.feature["image_raw"].bytes_list.value[0])

28
28
1
8


# row 한개씩 저장된 데이터 불러오기

In [42]:
# 이진 형태로 저장된 파일에서 1건씩 불러오는 과정
filename = os.path.join(save_dir_TFRecord_01, 'train_record.tfrecords')
record_iterator = tf.python_io.tf_record_iterator(filename)
example = tf.train.Example()

In [43]:
first = 0
image_list = [];label_list = [];width_list = [];height_list = [];
for seralized_img_example in record_iterator:
    first += 1
    example.ParseFromString(seralized_img_example)
    image_list.append(np.fromstring(example.features.feature['image_raw'].bytes_list.value[0], dtype=np.uint8).reshape((28, 28, -1)))
    label_list.append(example.features.feature['label'].int64_list.value[0])
    width_list.append(example.features.feature['width'].int64_list.value[0])
    height_list.append(example.features.feature['height'].int64_list.value[0])
image = np.array(image_list)
label = np.array(label_list)
width = np.array(width_list)
height = np.array(height_list)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


In [44]:
image.shape

(10, 28, 28, 1)

# 배열 한꺼번에 저장하기

In [62]:
data_sets[0]

In [100]:
print_onoff = 0
data_set = data_sets[0]
filename_array = os.path.join(save_dir_TFRecord_01, 'train_array.tfrecords')
print('filename : {}'.format(filename))
writer_array = tf.python_io.TFRecordWriter(filename_array)

label_bytes= data_set.labels[:10].tostring()
image_bytes= data_set.images[:10].tostring()
image_cnt = data_set.images[:10].shape[0]
image_width = 28
image_height = 28

example_array = tf.train.Example(features=tf.train.Features(feature={
 'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[label_bytes])),
 'image_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes])),
 'cnt': tf.train.Feature(int64_list=tf.train.Int64List(value=[image_cnt])), 
 'width': tf.train.Feature(int64_list=tf.train.Int64List(value=[image_width])), 
 'height': tf.train.Feature(int64_list=tf.train.Int64List(value=[image_height])), 
 }))


writer_array.write(example_array.SerializeToString())
writer_array.close()


filename : save_dir_TFRecord_01/train_array.tfrecords


# 한번에 저장된 데이터 불러오기

In [101]:
# 이진 형태로 저장된 파일에서 1건씩 불러오는 과정
filename = os.path.join(save_dir_TFRecord_01, 'train_array.tfrecords')
record_iterator = tf.python_io.tf_record_iterator(filename)
seralized_img_example = next(record_iterator)

example = tf.train.Example()
example.ParseFromString(seralized_img_example)

image = np.fromstring(example.features.feature['image_raw'].bytes_list.value[0], dtype=np.uint8).reshape(10,28,28,1)
label = np.fromstring(example.features.feature['label'].bytes_list.value[0], dtype=np.uint8)
cnt = example.features.feature['cnt'].int64_list.value[0]
width = example.features.feature['width'].int64_list.value[0]
height = example.features.feature['height'].int64_list.value[0]
